In [31]:
import requests
import json
import bs4
import pandas as pd
import numpy as np
import threading
from tqdm import tqdm

import time
import datetime

header ={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}

#cap=["nano","micro,"small,"medium,"large,"mega"]
exchange=["nasdaq","nyse","amex"]

In [32]:

def callGetAllCode(exchanges):
    stockCodes=[]
    for exg in exchanges:
        allStockAddr="https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=100000&exchange="+exg+"&marketcap=nano"
        allStockData=requests.get(allStockAddr,headers=header,timeout=10).text
        #print(allStockData)
        allStockJson=json.loads(allStockData)
        #print(allStockJson)
        allStockJson=allStockJson["data"]["table"]["rows"]
        for firm in allStockJson:
            stockCodes.append(firm["symbol"])
    return stockCodes
stockCodes=callGetAllCode(exchange)
print(stockCodes)

['BMR', 'ANEB', 'NSTS', 'HYMC', 'BNR', 'CETY', 'NISN', 'CLST', 'CPTN', 'BLRX', 'DWSN', 'AIRTP', 'FKWL', 'CRWS', 'CPBI', 'FLUX', 'LEE', 'SWVL', 'RRGB', 'RPHM', 'OABIW', 'CMBM', 'RGS', 'RECT', 'TTOO', 'PPSI', 'FLNT', 'MESA', 'BWEN', 'RLYB', 'HOUR', 'ARQQ', 'CREX', 'OSS', 'VTVT', 'TCBS', 'AWRE', 'EOSEW', 'LPTH', 'NTRB', 'DRRX', 'UCL', 'CCG', 'PFXNZ', 'HSON', 'BGM', 'PASG', 'LVTX', 'RVPH', 'EHGO', 'AADI', 'SYT', 'NICK', 'ICMB', 'CFSB', 'FSEA', 'ATRA', 'BGFV', 'OPHC', 'SELX', 'LVROW', 'OPTX', 'RAND', 'DXR', 'BCG', 'LUCD', 'EYEN', 'ASRV', 'YYGH', 'APYX', 'CMCT', 'ELTX', 'FATBW', 'SPAI', 'NAII', 'OCX', 'PDYN', 'TACT', 'PLBY', 'UHGWW', 'CARM', 'CTRM', 'ARKR', 'KZR', 'NAAS', 'INTG', 'CODX', 'PHUN', 'KPLT', 'TRSG', 'FBIO', 'ARKOW', 'IDN', 'USIO', 'SYRS', 'ECDA', 'RDI', 'RCON', 'MNY', 'PBBK', 'CDROW', 'HUIZ', 'FGEN', 'PODC', 'DGHI', 'HFBL', 'AKTX', 'FEAM', 'ECOR', 'DRCT', 'LINK', 'NXTC', 'RFIL', 'PET', 'ESLA', 'SWIN', 'CLIR', 'BNGO', 'CCLD', 'BHIL', 'RENT', 'MNDO', 'XOS', 'TARA', 'DSWL', 'EBON', 

In [33]:
def callGetStock(threads_numb,targetStocks,start_time,end_time,interval):
    yahoofinanceAPIBaseLink="https://query2.finance.yahoo.com/v8/finance/chart/"
    threads_num=threads_numb
    threads=[]
    fullStockInfo=[]
    splitStockCode=[]
    divNumb=int(len(targetStocks)/threads_num)
    for i in range(0,threads_num):
        if(i!=(threads_num-1)):
            splitStockCode.append(targetStocks[divNumb*i:divNumb*(i+1)])
        else:
            splitStockCode.append(targetStocks[divNumb*i:len(targetStocks)])

    def requestStockDataYahoo(code):
        yahoofinanceAPIUrl=yahoofinanceAPIBaseLink+code+"?"+"period1="+start_time+"&period2="+end_time+"&interval="+interval+"&includePrePost=true&lang=en-US&region=US"
        #print(yahoofinanceAPIUrl)
        stockData=requests.get(yahoofinanceAPIUrl,headers=header,timeout=5)
        fullStockInfo.append(stockData.text)
        #print(stockData)

    def worker(codes,thread_numb):
        progress_bar = tqdm(codes,desc=f"Thread {thread_numb} Progress", position=thread_numb, leave=True,dynamic_ncols=False)
        for code in codes:
            requestStockDataYahoo(code)
            progress_bar.update(1)

    for i in range(threads_num):
        thread=threading.Thread(target=worker,args=(splitStockCode[i],i))
        threads.append(thread)

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    return fullStockInfo


In [34]:
stock=""
start_time=""
end_time=""
interval="1m"

curtime=datetime.datetime.now()
prevday=curtime-datetime.timedelta(days=1)
unixCurtime=time.mktime(curtime.timetuple())
unixPrevtime=time.mktime(prevday.timetuple())
start_time=str(int(unixPrevtime-unixPrevtime%100))
end_time=str(int(unixCurtime-unixCurtime%100))
#print(start_time)
#print(end_time)

fullStockInfo=callGetStock(64,stockCodes,start_time,end_time,interval)




Thread 0 Progress:   0%|          | 0/29 [00:00<?, ?it/s]

















































































































































































Thread 0 Progress:   7%|▋         | 2/29 [00:00<00:02, 11.35it/s]










































































































































































Thread 0 Progress:  14%|█▍        | 4/29 [00:00<00:03,  8.23it/s]










































































































































































Thread 0 Progress:  17%|█▋        | 5/29 [00:00<00:03,  6.75it/s]










































































































































































Thread 0 Progress:  21%|██        | 6/29 [00:00<00:03,

In [35]:
possibleStockCodes=[]

for i in range(len(fullStockInfo)):
    indvStockJson=json.loads(fullStockInfo[i])
    #print(type(indvStockJson))
    regularMarketPrice=indvStockJson["chart"]["result"][0]["meta"]["regularMarketPrice"]
    if(regularMarketPrice<=1):
        symbol=indvStockJson["chart"]["result"][0]["meta"]["symbol"]
        possibleStockCodes.append(symbol)